This example uses the orange telecoms churn prediction data set as an example for binary classfication over deep learning framework. The first example uses a single layer netowrk while the second is a multilayer perceptron. The loss function used in both cases is cross entropy and activation function of the last layer is sigmoid and softmax. The accuracy of both models is very close with MLP offering no significant advantage over single layer.

In [14]:

import pandas as pd
import numpy as np
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from sklearn import datasets
from sklearn import metrics
from sklearn import model_selection

import tensorflow as tf

In [15]:
#manual one hot encoding in bash
#cp Downloads/churn-bigml-80.csv notebooks/tf-binclass/
#cd notebooks/tf-binclass/
#sed -i -e 's/No/0/g' churn-bigml-80.csv
#sed -i -e 's/Yes/1/g' churn-bigml-80.csv
#sed -i -e 's/True/1/g' churn-bigml-80.csv
#sed -i -e 's/False/0/g' churn-bigml-80.csv
#awk ' BEGIN{FS=OFS=","}NR>1{sub(/[^,]+,/,"");print}' churn-bigml-80.csv >> churn-bigml-80-1.csv


import numpy as np

def convertToOneHot(vector, num_classes=None):
    """
    Converts an input 1-D vector of integers into an output
    2-D array of one-hot vectors, where an i'th input value
    of j will set a '1' in the i'th row, j'th column of the
    output array.

    Example:
        v = np.array((1, 0, 4))
        one_hot_v = convertToOneHot(v)
        print one_hot_v

        [[0 1 0 0 0]
         [1 0 0 0 0]
         [0 0 0 0 1]]
    """

    assert isinstance(vector, np.ndarray)
    assert len(vector) > 0

    if num_classes is None:
        num_classes = np.max(vector)+1
    else:
        assert num_classes > 0
        assert num_classes >= np.max(vector)

    result = np.zeros(shape=(len(vector), num_classes))
    result[np.arange(len(vector)), vector] = 1
    return result.astype(int)

In [16]:
import sklearn.preprocessing

df=pd.read_csv("churn-bigml-80-1.csv", sep=',',header=None)
df2=df.iloc[:,-1]
df = df.iloc[:, :-1]
d=np.float32(df.values)
l=np.float32(df2.values)
l=np.float32(convertToOneHot(df2.values))

In [17]:
# Parameters
learning_rate = 0.000001
training_epochs = 20
batch_size = 100
display_step = 1

x = tf.placeholder(tf.float32, [None, 18])
W = tf.Variable(tf.zeros([18, 2]))
b = tf.Variable(tf.zeros([2]))
y = tf.matmul(x, W) + b

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 2])



In [8]:
# Construct model
pred = tf.nn.sigmoid(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1)) # 0 cost all over, except step 1
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels= y)) # 0 cost all over 
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels= y)) # 0 cost all over 

# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()



In [9]:
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = 1
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = d[1:1800,]
            batch_ys = l[1:1800,]           # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    # Calculate accuracy for remaining examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: d[1801:,], y: l[1801:,]}))

Epoch: 0001 cost= 0.693145990
Epoch: 0002 cost= 0.748973012
Epoch: 0003 cost= 0.815795422
Epoch: 0004 cost= 0.897080362
Epoch: 0005 cost= 0.997678638
Epoch: 0006 cost= 1.124360800
Epoch: 0007 cost= 1.286452651
Epoch: 0008 cost= 1.496533751
Epoch: 0009 cost= 1.771315813
Epoch: 0010 cost= 2.133111954
Epoch: 0011 cost= 2.612392902
Epoch: 0012 cost= 3.251469612
Epoch: 0013 cost= 4.108876705
Epoch: 0014 cost= 5.264389515
Epoch: 0015 cost= 6.825084209
Epoch: 0016 cost= 8.933474541
Epoch: 0017 cost= 11.779605865
Epoch: 0018 cost= 15.618833542
Epoch: 0019 cost= nan
Epoch: 0020 cost= nan
Optimization Finished!
Accuracy: 0.833526


It is our turn to use a multilayer perceptron. For this purpose I am using Keras, which is a deep learning library for Tensorflow and Theano. Kera makes it very easy to describe the architecture of multilayer perceptron and it is very hot in kaggle world.

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import *

model = Sequential()
model.add(Dense(10, input_dim=18, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss="binary_crossentropy" , optimizer='adagrad',
              metrics=['accuracy'])

model.fit(d[1:1800,], l[1:1800,1])

Epoch 1/10
1799/1799 [==============================] - 0s - loss: 1.8951 - acc: 0.7487      
Epoch 2/10
1799/1799 [==============================] - 0s - loss: 0.5990 - acc: 0.8482     
Ep

In [13]:
score = model.evaluate(d[1801:,], l[1801:,1])
score

 32/865 [>.............................] - ETA: 1s

[0.46850094970940165, 0.83583815028901731]